In [ ]:
# 쿠팡후기크롤링(엑셀구글드라이브저장버전)


import time
import logging
import pandas as pd
import os
import subprocess
import sys
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, JavascriptException
import pyautogui  # 새로 추가된 라이브러리

# Google Drive API 관련 라이브러리
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pickle

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def setup_driver():
    options = Options()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    return webdriver.Chrome(options=options)

def safe_find_text(element, by, value, default="-"):
    try:
        found_element = element.find_element(by, value)
        return found_element.text if found_element else default
    except NoSuchElementException:
        return default

def scrape_review(driver, review_element, index):
    review_data = {'순번': index + 1}
    
    logger.info("리뷰를 스크래핑하고 있습니다...")

    # 리뷰어 이름
    review_data['리뷰어_이름'] = safe_find_text(review_element, By.CSS_SELECTOR, ".sdp-review__article__list__info__user__name")

    # 리뷰어 배지 (예: Vine 프로그램)
    reviewer_badge = safe_find_text(review_element, By.CSS_SELECTOR, ".sdp-review__article__list__info__adventure-badge")
    review_data['체험단_리뷰어'] = 'VINE' in reviewer_badge

    # 별점
    star_element = review_element.find_element(By.CSS_SELECTOR, ".sdp-review__article__list__info__product-info__star-orange")
    style = star_element.get_attribute("style") if star_element else ""
    width = style.split("width:")[1].split("%")[0] if "width:" in style else "0"
    review_data['평점'] = float(width) / 20 if width else "-"

    # 리뷰 날짜 (yyyy.mm.dd -> yyyy-mm-dd 형식으로 변경)
    original_date = safe_find_text(review_element, By.CSS_SELECTOR, ".sdp-review__article__list__info__product-info__reg-date")
    review_data['날짜'] = original_date.replace('.', '-') if original_date != "-" else original_date

    # 판매자 이름
    review_data['판매자_이름'] = safe_find_text(review_element, By.CSS_SELECTOR, ".sdp-review__article__list__info__product-info__seller_name")

    # 제품명
    review_data['제품명'] = safe_find_text(review_element, By.CSS_SELECTOR, ".sdp-review__article__list__info__product-info__name")

    # 리뷰 제목
    review_data['리뷰제목'] = safe_find_text(review_element, By.CSS_SELECTOR, ".sdp-review__article__list__headline")

    # 리뷰 내용
    review_data['리뷰내용'] = safe_find_text(review_element, By.CSS_SELECTOR, ".sdp-review__article__list__review__content")

    # 도움이 된 수
    review_data['도움됨_수'] = safe_find_text(review_element, By.CSS_SELECTOR, ".js_reviewArticleHelpfulCount")

    logger.info("이 리뷰의 스크래핑을 완료했습니다.")
    return review_data

def scrape_reviews(url, reviews_to_crawl, filename_base, sort_option):
    if "www.coupang.com" not in url:
        logger.error("잘못된 URL입니다. www.coupang.com이 포함된 URL을 입력하세요.")
        return []

    driver = setup_driver()
    driver.get(url)
    reviews = []
    current_page = 1
    reviews_per_page = 5
    pages_to_crawl = -(-reviews_to_crawl // reviews_per_page)

    try:
        WebDriverWait(driver, 60).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )
        
        review_tab_script = """
        const reviewTab = document.querySelector('#pdpReviewContentTab, #btfTab > ul.tab-titles > li:nth-child(2)');
        if (reviewTab) {
            reviewTab.click();
            return true;
        }
        return false;
        """
        is_clicked = driver.execute_script(review_tab_script)
        
        if not is_clicked:
            logger.error("리뷰 탭을 찾을 수 없습니다.")
            return reviews
        
        if sort_option == 2:
            logger.info("최신순 버튼을 클릭하고 있습니다...")
            try:
                newest_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, ".sdp-review__article__order__sort__newest-btn.js_reviewArticleNewListBtn.js_reviewArticleSortBtn"))
                )
                newest_button.click()
                time.sleep(2)
            except TimeoutException:
                logger.warning("최신순 버튼을 찾을 수 없습니다. 기본 정렬 순서로 진행합니다.")
        
        retry_count = 0
        while len(reviews) < reviews_to_crawl and current_page <= pages_to_crawl:
            try:
                WebDriverWait(driver, 30).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, ".sdp-review__article__list"))
                )
            except TimeoutException:
                if retry_count < 3:  # 최대 3번 재시도
                    logger.warning("리뷰를 찾을 수 없습니다. Page Down을 시도합니다.")
                    for _ in range(7):
                        pyautogui.press('pgdn')
                        time.sleep(0.5)
                    retry_count += 1
                    continue
                else:
                    logger.error("리뷰를 찾을 수 없습니다. 다음 URL로 넘어갑니다.")
                    return reviews

            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)

            review_elements = driver.find_elements(By.CSS_SELECTOR, ".sdp-review__article__list")
            logger.info(f"페이지 {current_page}: {len(review_elements)}개의 리뷰를 찾았습니다")

            page_reviews = [scrape_review(driver, review_element, i + len(reviews)) for i, review_element in enumerate(review_elements)]
            reviews.extend(page_reviews[:reviews_to_crawl - len(reviews)])

            if len(reviews) % 100 == 0:
                save_to_csv(reviews, f"{filename_base}_중간저장_{len(reviews)}개.csv")
                logger.info(f"{len(reviews)}개의 리뷰가 중간 저장되었습니다.")

            if len(reviews) < reviews_to_crawl and current_page < pages_to_crawl:
                try:
                    if current_page % 10 == 0:
                        next_button = WebDriverWait(driver, 10).until(
                            EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.sdp-review__article__page__next'))
                        )
                    else:
                        next_button = WebDriverWait(driver, 10).until(
                            EC.element_to_be_clickable((By.CSS_SELECTOR, f'button.sdp-review__article__page__num[data-page="{current_page + 1}"]'))
                        )
                    driver.execute_script("arguments[0].click();", next_button)
                    current_page += 1
                    time.sleep(3)
                except (NoSuchElementException, TimeoutException) as e:
                    logger.info(f"더 이상 페이지가 없습니다. 페이지 {current_page}에서 중지합니다. 오류: {str(e)}")
                    break
            else:
                logger.info(f"지정된 리뷰 수({reviews_to_crawl})에 도달했습니다")
                break

    except TimeoutException as e:
        logger.error(f"페이지 로딩 시간이 초과되었습니다: {str(e)}")
    except JavascriptException as e:
        logger.error(f"JavaScript 실행 중 오류가 발생했습니다: {str(e)}")
    except Exception as e:
        logger.error(f"스크래핑 중 오류가 발생했습니다: {str(e)}")
    finally:
        driver.quit()

    return reviews[:reviews_to_crawl]

def save_to_csv(reviews, filename):
    df = pd.DataFrame(reviews)
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    logger.info(f"리뷰가 {filename} 파일로 저장되었습니다.")

def open_file(filename):
    if sys.platform.startswith('darwin'):  # macOS
        subprocess.call(('open', filename))
    elif sys.platform.startswith('win'):  # Windows
        os.startfile(filename)
    else:  # linux
        subprocess.call(('xdg-open', filename))

def upload_to_google_drive(file_path, folder_id):
    SCOPES = ['https://www.googleapis.com/auth/drive.file']
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('drive', 'v3', credentials=creds)

    file_metadata = {'name': os.path.basename(file_path), 'parents': [folder_id]}
    media = MediaFileUpload(file_path, resumable=True)
    file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    logger.info(f"파일이 구글 드라이브에 업로드되었습니다. File ID: {file.get('id')}")

def process_excel_input(excel_file):
    df = pd.read_excel(excel_file)
    
    # 엑셀 파일의 실제 열 이름을 출력
    print("엑셀 파일의 열 이름:", df.columns.tolist())
    
    # 열 이름 매핑 (필요한 경우 수정)
    column_mapping = {
        'URL': 'URL',  # 예: 엑셀의 'URL' 열은 그대로 'URL'
        '크롤링할 리뷰 수': '크롤링할 리뷰수',  # 예: 엑셀의 '크롤링할 리뷰 수' 열
        '파일 이름': '파일 이름',  # 예: 엑셀의 '파일 이름' 열
        '정렬 옵션': '정렬 옵션'  # 예: 엑셀의 '정렬 옵션' 열
    }
    
    for index, row in df.iterrows():
        url = row[column_mapping['URL']]
        reviews_to_crawl = int(row[column_mapping['크롤링할 리뷰 수']])
        filename_base = row[column_mapping['파일 이름']]
        sort_option = int(row[column_mapping['정렬 옵션']])

        if "www.coupang.com" not in url:
            logger.error(f"행 {index+2}: 잘못된 URL입니다. www.coupang.com이 포함된 URL을 입력하세요.")
            continue

        today_date = datetime.now().strftime("%Y%m%d")
        filename = f"{filename_base}_쿠팡후기_{'베스트순' if sort_option == 1 else '최신순'}_{today_date}.csv"

        scraped_reviews = scrape_reviews(url, reviews_to_crawl, filename_base, sort_option)

        if scraped_reviews:
            save_to_csv(scraped_reviews, filename)
            
            # Google Drive 업로드
            GOOGLE_DRIVE_FOLDER_ID = "11gWrvD22k-C_DDRedXQevZWnZ7bbN7Iv"  # 실제 폴더 ID로 변경해야 함
            try:
                upload_to_google_drive(filename, GOOGLE_DRIVE_FOLDER_ID)
                logger.info(f"파일 {filename}이 성공적으로 구글 드라이브에 업로드되었습니다.")
                
                # 로컬 파일 삭제
                os.remove(filename)
                logger.info(f"로컬 파일 {filename}이 삭제되었습니다.")
            except Exception as e:
                logger.error(f"구글 드라이브 업로드 중 오류가 발생했습니다: {str(e)}")
        else:
            logger.error(f"행 {index+2}: 리뷰를 스크래핑하지 못했습니다.")

if __name__ == "__main__":
    excel_file = input("크롤링할 정보가 담긴 엑셀 파일 이름을 입력하세요: ")
    process_excel_input(excel_file)
    
    # 크롤링할 엑셀 
    # C:\Users\owner\Documents\python\엑셀업로드양식(쿠팡).xlsx

In [9]:
# 쿠팡크롤링 통합 버전 (구글드라이브 저장)
import time
import logging
import pandas as pd
import os
import subprocess
import sys
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, JavascriptException

# Google Drive API 관련 라이브러리 추가
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pickle

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def setup_driver():
    options = Options()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    return webdriver.Chrome(options=options)

def safe_find_text(element, by, value, default="-"):
    try:
        found_element = element.find_element(by, value)
        return found_element.text if found_element else default
    except NoSuchElementException:
        return default

def scrape_review(driver, review_element, index):
    review_data = {'순번': index + 1}
    
    logger.info("리뷰를 스크래핑하고 있습니다...")

    # 리뷰어 이름
    review_data['리뷰어_이름'] = safe_find_text(review_element, By.CSS_SELECTOR, ".sdp-review__article__list__info__user__name")

    # 리뷰어 배지 (예: Vine 프로그램)
    reviewer_badge = safe_find_text(review_element, By.CSS_SELECTOR, ".sdp-review__article__list__info__adventure-badge")
    review_data['체험단_리뷰어'] = 'VINE' in reviewer_badge

    # 별점
    star_element = review_element.find_element(By.CSS_SELECTOR, ".sdp-review__article__list__info__product-info__star-orange")
    style = star_element.get_attribute("style") if star_element else ""
    width = style.split("width:")[1].split("%")[0] if "width:" in style else "0"
    review_data['평점'] = float(width) / 20 if width else "-"

    # 리뷰 날짜 (yyyy.mm.dd -> yyyy-mm-dd 형식으로 변경)
    original_date = safe_find_text(review_element, By.CSS_SELECTOR, ".sdp-review__article__list__info__product-info__reg-date")
    review_data['날짜'] = original_date.replace('.', '-') if original_date != "-" else original_date

    # 판매자 이름
    review_data['판매자_이름'] = safe_find_text(review_element, By.CSS_SELECTOR, ".sdp-review__article__list__info__product-info__seller_name")

    # 제품명
    review_data['제품명'] = safe_find_text(review_element, By.CSS_SELECTOR, ".sdp-review__article__list__info__product-info__name")

    # 리뷰 제목
    review_data['리뷰제목'] = safe_find_text(review_element, By.CSS_SELECTOR, ".sdp-review__article__list__headline")

    # 리뷰 내용
    review_data['리뷰내용'] = safe_find_text(review_element, By.CSS_SELECTOR, ".sdp-review__article__list__review__content")

    # 도움이 된 수
    review_data['도움됨_수'] = safe_find_text(review_element, By.CSS_SELECTOR, ".js_reviewArticleHelpfulCount")

    logger.info("이 리뷰의 스크래핑을 완료했습니다.")
    return review_data

def scrape_reviews(url, reviews_to_crawl, filename_base, sort_option):
    # URL 유효성 체크
    if "www.coupang.com" not in url:
        logger.error("잘못된 URL입니다. www.coupang.com이 포함된 URL을 입력하세요.")
        return []

    driver = setup_driver()
    driver.get(url)
    reviews = []
    current_page = 1
    reviews_per_page = 5  # 쿠팡의 경우 한 페이지당 5개의 리뷰가 있습니다
    pages_to_crawl = -(-reviews_to_crawl // reviews_per_page)  # 올림 나눗셈

    try:
        # 페이지가 완전히 로드될 때까지 대기
        WebDriverWait(driver, 60).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )
        
        # 리뷰 탭 클릭
        review_tab_script = """
        const reviewTab = document.querySelector('#pdpReviewContentTab, #btfTab > ul.tab-titles > li:nth-child(2)');
        if (reviewTab) {
            reviewTab.click();
            return true;
        }
        return false;
        """
        is_clicked = driver.execute_script(review_tab_script)
        
        if not is_clicked:
            logger.error("리뷰 탭을 찾을 수 없습니다.")
            return reviews
        
        # 정렬 옵션 선택
        if sort_option == 2:  # 최신순
            logger.info("최신순 버튼을 클릭하고 있습니다...")
            try:
                newest_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, ".sdp-review__article__order__sort__newest-btn.js_reviewArticleNewListBtn.js_reviewArticleSortBtn"))
                )
                newest_button.click()
                time.sleep(2)
            except TimeoutException:
                logger.warning("최신순 버튼을 찾을 수 없습니다. 기본 정렬 순서로 진행합니다.")
        
        while len(reviews) < reviews_to_crawl and current_page <= pages_to_crawl:
            WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".sdp-review__article__list"))
            )
            
            # 스크롤 다운
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)

            review_elements = driver.find_elements(By.CSS_SELECTOR, ".sdp-review__article__list")
            logger.info(f"페이지 {current_page}: {len(review_elements)}개의 리뷰를 찾았습니다")

            # 순번 추가
            page_reviews = [scrape_review(driver, review_element, i + len(reviews)) for i, review_element in enumerate(review_elements)]
            reviews.extend(page_reviews[:reviews_to_crawl - len(reviews)])  # 필요한 만큼만 추가

            if len(reviews) % 100 == 0:
                save_to_csv(reviews, f"{filename_base}_중간저장_{len(reviews)}개.csv")
                logger.info(f"{len(reviews)}개의 리뷰가 중간 저장되었습니다.")

            if len(reviews) < reviews_to_crawl and current_page < pages_to_crawl:
                try:
                    if current_page % 10 == 0:
                        next_button = WebDriverWait(driver, 10).until(
                            EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.sdp-review__article__page__next'))
                        )
                    else:
                        next_button = WebDriverWait(driver, 10).until(
                            EC.element_to_be_clickable((By.CSS_SELECTOR, f'button.sdp-review__article__page__num[data-page="{current_page + 1}"]'))
                        )
                    driver.execute_script("arguments[0].click();", next_button)
                    current_page += 1
                    time.sleep(3)
                except (NoSuchElementException, TimeoutException) as e:
                    logger.info(f"더 이상 페이지가 없습니다. 페이지 {current_page}에서 중지합니다. 오류: {str(e)}")
                    break
            else:
                logger.info(f"지정된 리뷰 수({reviews_to_crawl})에 도달했습니다")
                break

    except TimeoutException as e:
        logger.error(f"페이지 로딩 시간이 초과되었습니다: {str(e)}")
    except JavascriptException as e:
        logger.error(f"JavaScript 실행 중 오류가 발생했습니다: {str(e)}")
    except Exception as e:
        logger.error(f"스크래핑 중 오류가 발생했습니다: {str(e)}")
    finally:
        
        # driver.quit()
        pass

    return reviews[:reviews_to_crawl]

def save_to_csv(reviews, filename):
    df = pd.DataFrame(reviews)
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    logger.info(f"리뷰가 {filename} 파일로 저장되었습니다.")

def open_file(filename):
    if sys.platform.startswith('darwin'):  # macOS
        subprocess.call(('open', filename))
    elif sys.platform.startswith('win'):  # Windows
        os.startfile(filename)
    else:  # linux
        subprocess.call(('xdg-open', filename))

# Google Drive 업로드 함수
def upload_to_google_drive(file_path, folder_id):
    SCOPES = ['https://www.googleapis.com/auth/drive.file']
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('drive', 'v3', credentials=creds)

    file_metadata = {'name': os.path.basename(file_path), 'parents': [folder_id]}
    media = MediaFileUpload(file_path, resumable=True)
    file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    logger.info(f"파일이 구글 드라이브에 업로드되었습니다. File ID: {file.get('id')}")

if __name__ == "__main__":
    url = input("크롤링할 상품 URL을 입력하세요: ")
    if "www.coupang.com" not in url:
        print("잘못된 URL입니다. www.coupang.com이 포함된 URL을 입력하세요.")
        sys.exit(1)
    
    reviews_to_crawl = int(input("크롤링할 리뷰 수를 입력하세요: "))
    filename_base = input("저장할 파일 이름을 입력하세요 (확장자 제외): ")
    sort_option = int(input("정렬 옵션을 선택하세요 (1: 베스트순, 2: 최신순): "))
    
    today_date = datetime.now().strftime("%Y%m%d")
    filename = f"{filename_base}_쿠팡후기_{'베스트순' if sort_option == 1 else '최신순'}_{today_date}.csv"
    
    scraped_reviews = scrape_reviews(url, reviews_to_crawl, filename_base, sort_option)

    if scraped_reviews:
        save_to_csv(scraped_reviews, filename)
        open_file(filename)
        
        # Google Drive 업로드
        GOOGLE_DRIVE_FOLDER_ID = "11gWrvD22k-C_DDRedXQevZWnZ7bbN7Iv"  # 여기에 실제 폴더 ID를 입력하세요
        try:
            upload_to_google_drive(filename, GOOGLE_DRIVE_FOLDER_ID)
            logger.info("파일이 성공적으로 구글 드라이브에 업로드되었습니다.")
            
             # 로컬 파일 삭제
            os.remove(filename)
            logger.info(f"로컬 파일 {filename}이 삭제되었습니다.")
        except Exception as e:
            logger.error(f"구글 드라이브 업로드 중 오류가 발생했습니다: {str(e)}")
    else:
        print("리뷰를 스크래핑하지 못했습니다.")

IndentationError: expected an indented block after 'finally' statement on line 175 (203942631.py, line 179)